In [17]:
from sksurv.datasets import load_breast_cancer

In [18]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter

from sklearn import linear_model
#from sklearn.linear_model import LassoCV
#from sklearn.model_selection import cross_val_score

from sksurv.linear_model import CoxPHSurvivalAnalysis 

from sksurv.linear_model import  CoxnetSurvivalAnalysis
## elastic net but can be applyed wuth lasso

from lifelines.utils import concordance_index
from lifelines.utils import k_fold_cross_validation
from lifelines import CoxPHFitter

from sklearn.model_selection import GridSearchCV

## hot one encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

## Scoring
from sklearn.metrics import make_scorer
from lifelines.utils import concordance_index

In [19]:
#torch.arange(10)

In [20]:
# 

In [21]:
os.getcwd()

'/home/ronin/Dev/miniconda3/notebooks/thesis_notebook'

In [22]:
data = pd.read_csv("processed/BLCA_data_preprocessed.csv")

In [23]:
modality_list = ["clinical",
"gex",
"rppa",
"mirna",
"mutation",
"meth",
"cnv"]

In [25]:
omic = 0 # 0 is clinical
modalities_to_keep = modality_list[omic]
modality_mask = [col for col in data.columns if col.rsplit("_")[0] in modalities_to_keep]

In [26]:
X = data[modality_mask]
y = data[['OS', 'OS_days']].rename(columns={'OS_days' : 'time'}).rename(columns={'OS' : 'event'})
y['event'] = y['event'].astype(bool)
y = np.array(y.to_records(index=False))

In [1]:
X["clinical_gender"]

NameError: name 'X' is not defined

In [28]:
X["clinical_clinical_stage"].unique()

array([nan])

In [29]:
X = data[modality_mask]

In [30]:
X.dtypes

clinical_age_at_initial_pathologic_diagnosis      int64
clinical_gender                                  object
clinical_race                                    object
clinical_ajcc_pathologic_tumor_stage             object
clinical_clinical_stage                         float64
clinical_histological_type                       object
dtype: object

In [129]:
#X["clinical_gender"].unique()

In [130]:
#X["clinical_race"].unique()

In [31]:
if omic == 0:
    ct = ColumnTransformer(
        [
            (
                "numerical",
                make_pipeline(StandardScaler()),
                np.where(X.dtypes != "object")[0],
            ),
            (
                "categorical",
                make_pipeline(
                    OneHotEncoder(
                        sparse=False, handle_unknown="ignore"
                    ),
                    StandardScaler(),
                ),
                np.where(X.dtypes == "object")[0],
            ),
        ]
    )
else:
    ct = ColumnTransformer(
        [
            (
                "numerical",
                make_pipeline(StandardScaler()),
                np.where(X_train.dtypes != "object")[0],
            )
        ]
    )

In [32]:
Xt = pd.DataFrame(ct.fit_transform(X)).fillna(0)
Xt

/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.271433,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,-0.731357,1.414214,-0.078087,0.095783,-0.095783
1,-0.650176,0.0,1.739077,-1.739077,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,-0.731357,1.414214,-0.078087,0.095783,-0.095783
2,1.054169,0.0,1.739077,-1.739077,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,1.367320,-0.707107,-0.078087,0.095783,-0.095783
3,0.296682,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,1.367320,-0.707107,-0.078087,0.095783,-0.095783
4,-0.744862,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,1.505765,-0.731357,-0.707107,-0.078087,0.095783,-0.095783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,-0.271433,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,1.367320,-0.707107,-0.078087,0.095783,-0.095783
326,1.432913,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,1.505765,-0.731357,-0.707107,-0.078087,0.095783,-0.095783
327,-0.839548,0.0,1.739077,-1.739077,-0.360745,-0.24717,-1.927248,4.938078,-0.078087,-0.664114,1.367320,-0.707107,-0.078087,0.095783,-0.095783
328,0.107311,0.0,-0.575018,0.575018,-0.360745,-0.24717,0.518875,-0.202508,-0.078087,-0.664114,1.367320,-0.707107,-0.078087,0.095783,-0.095783


In [47]:
Xt = pd.DataFrame(OneHotEncoder().fit_transform(X))

In [133]:
column1 = Xt[Xt.columns[0]]
mean = round(column1.mean(),5)
std = round(column1.std(),5)
print("mean: ", mean, "; std = ", std)

mean:  -0.0 ; std =  1.00152


In [134]:
Xt.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
954    float64
955    float64
956    float64
957    float64
958    float64
Length: 959, dtype: object

In [105]:
param_grid = {
    'l1_ratio': [0.01]*15, # 0 è L2, 1 è L1   [0.1, 0.5, 0.9],  # Specify a range of l1_ratio values to search over
    'penalty_factor': [np.logspace(-3, 3, 15)]  # Specify a range of penalty_factor values to search over
}

## Pipeline del metodo (nested CV)

- (?) vedi metodo automatico PredefinedSplit:

coeff
- for(k split prima divisione a 5 outer split)
    - gridSearchCV() oppure a mano (inner altri 5)
        - preproc -> va fittato nel training 
        - gridsearchCV coxNet sul train set (4/5), -> best lambda 
    - evluation sul test (concordance) (1/5),


1) prendo train/test split
    X, y-> Xtrain,Ytrain, Xtest,Ytest
    
(gridSearchCV)->average lamda

2) Metodo(hparams).fit(Xtrain, ytrain) -> comprende anche preproc, fitta anche preproc

3) scorer(metodo.predict(Xtest), ytest)

    

In [106]:
coxnet = CoxnetSurvivalAnalysis()

In [107]:
y.dtype

dtype((numpy.record, [('event', '?'), ('time', '<i8')]))

In [108]:
#event_times (iterable) – a length-n iterable of observed survival times.
#predicted_scores (iterable) – a length-n iterable of predicted scores - these could be survival times, or hazards, etc. See https://stats.stackexchange.com/questions/352183/use-median-survival-time-to-calculate-cph-c-statistic/352435#352435
#event_observed (iterable, optional) – a length-n iterable censoring flags, 1 if observed, 0 if not. Default None assumes all observed.

def scorer2 (model, X, y):
    rev_y = np.rec.fromrecords(y)
    y_censorings = rev_y[0].view()
    y_times = rev_y[1].view()
    y_predicted = model.predict(X)
    concordance_index(y_predicted, y_times, y_censorings)
    
#in make scorer score_funccallable
#Score function (or loss function) with signature score_func(y, y_pred, **kwargs).



In [109]:
def scorer3 (y, y_pred):
    y_censorings = list( np.rec.fromrecords(y)[0].view())
    y_times = list( np.rec.fromrecords(y)[1].view())
    return concordance_index(y_times, y_pred, y_censorings)

In [110]:
grid_search = GridSearchCV(coxnet, param_grid, cv = 5, scoring = make_scorer(scorer3), refit=True)
grid_search.fit(Xt, y)

GridSearchCV(cv=5, estimator=CoxnetSurvivalAnalysis(),
             param_grid={'l1_ratio': [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                                      0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                                      0.01],
                         'penalty_factor': [array([1.00000000e-03, 2.68269580e-03, 7.19685673e-03, 1.93069773e-02,
       5.17947468e-02, 1.38949549e-01, 3.72759372e-01, 1.00000000e+00,
       2.68269580e+00, 7.19685673e+00, 1.93069773e+01, 5.17947468e+01,
       1.38949549e+02, 3.72759372e+02, 1.00000000e+03])]},
             scoring=make_scorer(scorer3))

In [111]:
gridsearch()

NameError: name 'gridsearch' is not defined

In [63]:
scoring = make_scorer(
                    scorer3,
                    greater_is_better=True,
                    )

In [64]:
grid_search.cv_results_

{'mean_fit_time': array([0.00748811, 0.01270957, 0.00448699, 0.00378599, 0.00377278,
        0.00427589, 0.00394621, 0.00378799, 0.00379381, 0.00379171,
        0.00379057, 0.00378666, 0.00375657, 0.00379782, 0.00377183]),
 'std_fit_time': array([1.79075413e-03, 1.10747694e-02, 1.31449728e-03, 5.60389367e-05,
        5.48307582e-05, 1.05362753e-03, 1.26983141e-04, 7.50926936e-05,
        7.56959427e-05, 4.88050693e-05, 6.69893248e-05, 6.31548450e-05,
        6.12141547e-05, 9.50686304e-05, 9.10041437e-05]),
 'mean_score_time': array([0.0053575 , 0.00602393, 0.0026217 , 0.00212445, 0.00210376,
        0.00274811, 0.00226407, 0.00209994, 0.0021121 , 0.00210223,
        0.00211067, 0.00210638, 0.00208459, 0.00213242, 0.00209303]),
 'std_score_time': array([1.03707887e-03, 3.39632910e-03, 1.07394221e-03, 5.43323667e-05,
        2.97385105e-05, 1.09644866e-03, 2.11762808e-04, 3.91016355e-05,
        5.10806784e-05, 4.94295479e-05, 3.91611933e-05, 6.00020322e-05,
        4.40801330e-05, 7.39

In [33]:
CoxnetSurvivalAnalysis(**grid_search.best_params_)

CoxnetSurvivalAnalysis(l1_ratio=0.01,
                       penalty_factor=array([1.00000000e-03, 2.68269580e-03, 7.19685673e-03, 1.93069773e-02,
       5.17947468e-02, 1.38949549e-01, 3.72759372e-01, 1.00000000e+00,
       2.68269580e+00, 7.19685673e+00, 1.93069773e+01, 5.17947468e+01,
       1.38949549e+02, 3.72759372e+02, 1.00000000e+03]))

In [34]:
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio=0.01)
cox_lasso.fit(Xt, y)

CoxnetSurvivalAnalysis(alpha_min_ratio=0.01, l1_ratio=1.0)

In [ ]:
# per avere lo score potrei # cox_lasso.score(X, y)

In [71]:
#help(k_fold_cross_validation)

In [49]:
#help(concordance_index)

In [ ]:
scores = k_fold_cross_validation(lasso, 
                                 df = pd.concat([y, X], axis=1), 
                                 duration_col = "OS_days", 
                                 event_col = "OS", 
                                 scoring_method='concordance_index')

NameError: name 'lasso' is not defined

In [32]:
best_delta = lasso.alphas_[np.argmax(np.mean(scores, axis=0))]

AttributeError: 'LassoCV' object has no attribute 'alphas_'

# Note da FRA

In [57]:
# Assuming X is your feature matrix, y is your survival data with columns 'time' and 'event'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [58]:
y_train

,OS,OS_days
75,1,536
184,1,739
17,1,680
24,0,2293
180,1,168
...,...,...
188,0,384
71,0,2312
106,0,1174
270,0,373


In [17]:
lasso_cph = CoxPHFitter(penalizer=0.1)  # You can adjust the penalizer value for regularization
lasso_cph.fit(X_train_scaled, y_train, duration_col='time', event_col='event')

NameError: name 'X_train_scaled' is not defined

In [ ]:
# Predict survival probability for the test set
y_pred = lasso_cph.predict_survival_function(X_test_scaled)

# Calculate the C-index
c_index = concordance_index(y_test['time'], -y_pred.mean(axis=1), y_test['event'])
print("C-index: ", c_index)


In [36]:
from sklearn.model_selection import GridSearchCV

In [62]:
def my_custom_loss_func(y_true, y_pred):
    ...
    return concordance

,OS,OS_days
75,1,536
184,1,739
17,1,680
24,0,2293
180,1,168
...,...,...
188,0,384
71,0,2312
106,0,1174
270,0,373


In [65]:
cv = GridSearchCV(
    CoxPHSurvivalAnalysis(), 
    param_grid={'alpha':[0,1]},
    scoring=my_scoring_function
).fit(X_train, y_train.astype({'OS': bool}).to_records(index=False)))

/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:179: RuntimeWarning: divide by zero encountered in log
  loss -= (numerator - n_events * np.log(risk_set)) / n_samples
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:173: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:170: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:179: RuntimeWarning: invalid value encountered in double_scalars
  loss -= (numerator - n_events * np.log(risk_set)) / n_samples
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.py:194: RuntimeWarning: overflow encountered in exp
  exp_xw = np.exp(offset + np.dot(x, w))
/home/ronin/Dev/miniconda3/lib/python3.10/site-packages/sksurv/linear_model/coxph.

In [ ]:
from sklearn.model_selection import cross_val_predict

# Caso Priority Lasso
cross_val_predict(
    estimator=GridSearchCV(
        CoxPHSurvivalAnalysis(), 
        param_grid={'alpha':[0,1]},
        scoring=my_scoring_function,
        cv=#
    ),
    X=X_train,
    y=y_train
)

# Caso nostro
cross_val_predict(
    estimator=CoxPHSurvivalAnalysis(**cv._best_params_),
    X=X_train,
    y=y_train
)

In [69]:
cv.best_params_

{'alpha': 1}

In [ ]:
*args